# Data Collection

The purpose of this notebook is to get the data from kaggle.
Since the dataset is about 200 000 images, but we only need a fraction of that, this notebook will also be responsible for removing unnecessary data and structuring the files.

_Dataset is already split up into train, test and validation, however we merge all into one and split our data ourselves at a later stage_

In [21]:
import os
import shutil

# Vars
DATASET = 'ashishjangra27/gender-recognition-200k-images-celeba'
SUBSET_IMGS = 3000
ROOT_DIR = os.path.dirname(os.getcwd())

# Set kaggle.json path
os.environ['KAGGLE_CONFIG_DIR'] = ROOT_DIR

In [22]:
ROOT_DIR

'/Users/linuselvius/Documents/CI/genderpredictor/genderpredictor'

In [23]:
import kaggle

# Authenticate
kaggle.api.authenticate()

# Download dataset
kaggle.api.dataset_download_files(DATASET, path=ROOT_DIR, unzip=True)

In [24]:
dataset_dir = os.path.join(ROOT_DIR, 'Dataset')  # Dataset folder path
dirs = [dir for dir in os.listdir(dataset_dir)]  # ['Train', 'Test', 'Validation']
male_dirs = []
female_dirs = []

# Create lists of all directory paths
for dir in dirs:
    current_dirs = os.listdir(os.path.join(dataset_dir, dir))
    for child in current_dirs:
        if child == 'Male':
            male_dirs.append(os.path.join(dataset_dir, dir, child))
        elif child == 'Female':
            female_dirs.append(os.path.join(dataset_dir, dir, child))

In [25]:
male_dirs

['/Users/linuselvius/Documents/CI/genderpredictor/genderpredictor/Dataset/Test/Male',
 '/Users/linuselvius/Documents/CI/genderpredictor/genderpredictor/Dataset/Train/Male',
 '/Users/linuselvius/Documents/CI/genderpredictor/genderpredictor/Dataset/Validation/Male']

In [27]:
# Create new directories

main_dir = 'temp_dataset'

os.mkdir(os.path.join(ROOT_DIR, main_dir))

male_dir = 'male'
female_dir = 'female'

os.mkdir(os.path.join(ROOT_DIR, main_dir, male_dir))
os.mkdir(os.path.join(ROOT_DIR, main_dir, female_dir))

In [28]:
# Copy number of files (SUBSET_IMGS) to new directories
def copy_files() -> None:
    global male_dirs
    global female_dirs
    global ROOT_DIR
    global main_dir
    global dataset_dir
    subset_size = round(SUBSET_IMGS / len(male_dirs + female_dirs))

    for dir in male_dirs:
        index = 0
        for file in os.listdir(dir):
            if index == subset_size:
                break
            shutil.copy2(os.path.join(dir, file), os.path.join(ROOT_DIR, main_dir, 'male'))
            index += 1
    
    for dir in female_dirs:
        index = 0
        for file in os.listdir(dir):
            if index == subset_size:
                break
            shutil.copy2(os.path.join(dir, file), os.path.join(ROOT_DIR, main_dir, 'female'))
            index += 1

copy_files()

In [29]:
# Remove original dataset

shutil.rmtree(dataset_dir)

In [1]:
# Check size of new data
male_size = len(os.listdir(os.path.join(main_dir, 'male')))
female_size = len(os.listdir(os.path.join(main_dir, 'female')))

print(f'Male dir: {male_size} images')
print(f'Female dir: {female_size} images')
print(f'Total: {male_size + female_size} images')

NameError: name 'os' is not defined

In [ ]:
# Rename temp_dataset to dataset
os.rename(os.path.join(ROOT_DIR, main_dir), os.path.join(ROOT_DIR, 'dataset'))